In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC
import os
import urllib ## lets us download images

import pickle as pk
import genanki
import time

## Example from Mountaineering Club

In [ ]:

profile = webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList", 2)
# profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", "/home/ashley/Local Scripts/BIRDS/downloads")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")

anu_account = "organisation"

URL = "https://media.ebird.org/catalog?searchField=region&regionCode=AU&mediaType=photo&sort=rating_rank_desc&taxonCode=rebfir1"

with webdriver.Firefox(options=profile) as driver:

    driver.get(URL)
    button1 = driver.find_element("id","organisation")
    button1.click()
    # wait.until(presence_of_element_located((By.CSS_SELECTOR, "h3>a")))

    WebDriverWait(driver,10).until(
        EC.presence_of_element_located(("id","username"))
    )

    driver.find_element("id","username").send_keys("u6561094")
    driver.find_element("id","password").send_keys("rvZq&Rd9ZobG2xp*Z794")
    driver.find_element("name","_eventId_proceed").click()


    WebDriverWait(driver,10).until(
        EC.presence_of_element_located(("id","export-button-clone"))
    )

    driver.find_element("id","export-button-clone").click()

    time.sleep(2)
    driver.switch_to.alert.accept()
    time.sleep(2)

    p = driver.current_window_handle
    for window in driver.window_handles:
        if window != p:
            driver.switch_to.window(window)
    time.sleep(12)


    jobid = driver.current_url.split("/")[-1]
    print(driver.current_url)

    # result = driver.get(f"https://sportandwellbeing.anu-sport.com.au/Account/Jobs/File/{jobid}")


    # WebDriverWait(driver,15).until(
    #     EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div[2]/div[2]/div[2]/div[6]/div/div/a"))
    # )
    # button = driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[2]/div[2]/div[6]/div/div/a")

    WebDriverWait(driver,15).until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT,"Download"))
    )
    button = driver.find_element(By.PARTIAL_LINK_TEXT,"Download")
    button.click()



NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:436:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElement@chrome://remote/content/marionette/driver.sys.mjs:1480:15
despatch@chrome://remote/content/marionette/server.sys.mjs:303:40
execute@chrome://remote/content/marionette/server.sys.mjs:276:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:249:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:250:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:498:20


Basically, we need a list of all of the bird codes and names. Then, we just plug in the media search url with the right taxonCode query:

https://media.ebird.org/catalog?searchField=region&regionCode=AU&mediaType=photo&sort=rating_rank_desc&taxonCode=**rebfir1**

## Get list of bird names and taxonomy ids..

In [ ]:
profile = webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList", 2)
# profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", "/home/ashley/Local Scripts/BIRDS/downloads")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")

anu_account = "organisation"

URL = "https://ebird.org/region/AU?yr=all"

bird_dict = {}

with webdriver.Firefox(options=profile) as driver:

    driver.get(URL)

    # all_birds = driver.find_elements(By.TAG_NAME,"a")

    ## This is the class that contains all of the 'bird' boxes on this page. Need to inspect all of the elements under this class
    all_birds = driver.find_elements(By.CLASS_NAME,"Observation-species")

    # FUNCTION USED TO GET ALL ATTRBUTES OF AN ELEMENT!! VERY HELPFUL
    # print(driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', all_birds))


    ## Iterate through all of the elements with the appropriate class..
    for i in all_birds:

        ## Find the elements within this element that have the 'a' tag. This gives us the bird's taxonomy id
        a = i.find_elements(By.TAG_NAME,"a")

        ## likewise, the span class contains the name of the bird
        heading = i.find_elements(By.TAG_NAME,"span")

        ## Collect the attrbutes we want from these elements
        for j in a:
            tax_id = j.get_attribute("href")

        for j in heading:
            name = j.text

        ## If this is in fact the right element, save the corresponding bird names and ids into a dictionary
        if "https://ebird.org/species/" in tax_id:

            tax_id = tax_id.split("https://ebird.org/species/")[-1].split("/")[0]

            if "Exotic" not in name:
                bird_dict[tax_id] = name


pk.dump(bird_dict,open("bird_dict","wb"))

print(bird_dict)

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:436:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.sys.mjs:1525:15
despatch@chrome://remote/content/marionette/server.sys.mjs:303:40
execute@chrome://remote/content/marionette/server.sys.mjs:276:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:249:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:250:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:498:20


## For every bird in the dictionary, collect information from ebird about ID and status

In [13]:
bird_dict = pk.load(open("bird_dict","rb"))
profile = webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList", 2)
# profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", "/home/ashley/Local Scripts/BIRDS/downloads")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")

with webdriver.Firefox(options=profile) as driver:
    URL = f"https://ebird.org/map/succoc"
    driver.get(URL)
    pwd = "C8l#mwwp$SdukWa6TnQrsxU#t"
    acc = "ashjbarnes"
    driver.find_element("id","input-user-name").send_keys(acc)
    driver.find_element("id","input-password").send_keys(pwd)
    driver.find_element("id","form-submit").click()


    for taxid in bird_dict:
        print(taxid,bird_dict[taxid])
        # URL = f"https://ebird.org/species/{taxid}"
        # driver.get(URL)
        # element = driver.find_elements(By.ID,"conservation-status")

        # for i in element:
        #     try:
        #         print(i.find_element(By.CLASS_NAME,"Badge").text)
        #     except:
        #         print("notfound")
        #         pass


        # element = driver.find_elements(By.CLASS_NAME,"u-stack-sm")

        # for i in element:
        #     try:
        #         print(i.text)
        #     except:
        #         print("notfound")
        #         pass

        ### GET THE RANGE MAP
        URL = f"https://ebird.org/map/{taxid}"
        driver.get(URL)

        search =  WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"search-city")))

        search.send_keys("Australia")
        time.sleep(1)
        WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"Suggest-suggestion-0"))).click()
        # search.send_keys(Keys.RETURN)

        negtoggle = WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"negToggle")))
        negtoggle.click()

        map = WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"map-main")))
        time.sleep(4)
        map.screenshot(f"/home/ashley/Local_Scripts/BIRDS/downloads/map_{taxid}.png")



asikoe3 Pacific Koel
chbcuc2 Channel-billed Cuckoo
silgul2 Silver Gull
souboo8 Southern Boobook
litcor2 Little Corella
tawfro1 Tawny Frogmouth
auonig1 Australian Owlet-nightjar
maslap1 Masked Lapwing
dollar1 Dollarbird
angpet1 Southern Giant-Petrel
galah Galah
succoc Sulphur-crested Cockatoo
noimin1 Noisy Miner
redwat1 Red Wattlebird
eursta European Starling
criros2 Crimson Rosella
crepig1 Crested Pigeon
puclor1 Purple-crowned Lorikeet
muslor1 Musk Lorikeet
railor5 Rainbow Lorikeet
ausmag2 Australian Magpie
pieoys1 Pied Oystercatcher
hooplo2 Hooded Plover


NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:436:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElement@chrome://remote/content/marionette/driver.sys.mjs:1480:15
despatch@chrome://remote/content/marionette/server.sys.mjs:303:40
execute@chrome://remote/content/marionette/server.sys.mjs:276:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:249:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:250:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:498:20
observe@resource://gre/modules/AsyncShutdown.jsm:574:16


In [21]:
pwd

'/home/ashley/Local_Scripts/BIRDS'

In [ ]:
element

<selenium.webdriver.remote.webelement.WebElement (session="7c98bfd3-2d66-477b-96d4-f5b1edc104ad", element="b7a464d2-b20c-45ce-a3b4-7a27a1989418")>

## Now that we have a list of bird names and taxonomies, search for them all on the media page. This could be slow 

In [ ]:
profile = webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList", 2)
# profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", "/home/ashley/Local Scripts/BIRDS/downloads")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")
# profile.add_argument("--headless")

def download_image(url,name):

    ## Fix spaces in name
    tmp = name.split(" ")
    if len(tmp) > 1:
        name = tmp[0]
        for i in tmp[1:]:
            name += "\ " + i

    if url[-3:] == "480":
        url = url.split("480")[0] + "1200"
    os.system(f"wget -O /home/ashley/Local_Scripts/BIRDS/downloads/{name} {url} ")


def download_audio(url,name):

    ## Fix spaces in name
    tmp = name.split(" ")
    if len(tmp) > 1:
        name = tmp[0]
        for i in tmp[1:]:
            name += "\ " + i

    ## Replace "Poster" with "audio"

    url = url.split("poster")[0] + "audio"

    os.system(f"wget -O /home/ashley/Local_Scripts/BIRDS/downloads/{name} {url} ")


def fetch_birds(bird_dict):

    downloaded = os.listdir("downloads")

    with webdriver.Firefox(options=profile) as driver:
        for taxid in bird_dict:
            name = bird_dict[taxid]

            if f"{name}.png" not in downloaded:
                print(taxid,name)

                URL = f"https://media.ebird.org/catalog?searchField=region&regionCode=AU&mediaType=photo&sort=rating_rank_desc&taxonCode={taxid}"


                driver.get(URL)

                img_objects = driver.find_element(By.TAG_NAME,"img")

                download_image(img_objects.get_attribute("src"),f"{name}.png")


                URL = f"https://media.ebird.org/catalog?searchField=region&regionCode=AU&mediaType=audio&sort=rating_rank_desc&taxonCode={taxid}"

                driver.get(URL)
                time.sleep(5)
                audio_objects = driver.find_element(By.CLASS_NAME,"ResultsGallery-image")
                href = audio_objects.get_attribute("src")
                download_audio(href,f"{name}.mp3")


    return


fetch_birds(bird_dict)

lewhon1 Lewin's Honeyeater
slbblc1 Carnaby's Black-Cockatoo
hobcuc1 Horsfield's Bronze-Cuckoo
gilwhi1 Gilbert's Whistler
soppet1 Soft-plumaged Petrel
whhpet1 White-headed Petrel


NoSuchElementException: Message: Unable to locate element: .ResultsGallery-image
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:394:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:280:16


In [ ]:
os.list

In [ ]:
bird_dict

{'asikoe3': 'Pacific Koel',
 'chbcuc2': 'Channel-billed Cuckoo',
 'silgul2': 'Silver Gull',
 'souboo8': 'Southern Boobook',
 'litcor2': 'Little Corella',
 'tawfro1': 'Tawny Frogmouth',
 'auonig1': 'Australian Owlet-nightjar',
 'maslap1': 'Masked Lapwing',
 'dollar1': 'Dollarbird',
 'angpet1': 'Southern Giant-Petrel',
 'galah': 'Galah',
 'succoc': 'Sulphur-crested Cockatoo',
 'noimin1': 'Noisy Miner',
 'redwat1': 'Red Wattlebird',
 'eursta': 'European Starling',
 'criros2': 'Crimson Rosella',
 'crepig1': 'Crested Pigeon',
 'puclor1': 'Purple-crowned Lorikeet',
 'muslor1': 'Musk Lorikeet',
 'railor5': 'Rainbow Lorikeet',
 'ausmag2': 'Australian Magpie',
 'pieoys1': 'Pied Oystercatcher',
 'hooplo2': 'Hooded Plover',
 'whikit1': 'Whistling Kite',
 'wbseag1': 'White-bellied Sea-Eagle',
 'aukpar1': 'Australian King-Parrot',
 'gryfan1': 'Gray Fantail',
 'chetea1': 'Chestnut Teal',
 'grecor': 'Great Cormorant',
 'auspel1': 'Australian Pelican',
 'laukoo1': 'Laughing Kookaburra',
 'sackin1': 'S

In [ ]:
bird_dict = pk.load(open("bird_dict","rb"))

i = 0
for taxid in bird_dict:
    fetch_birds(taxid,bird_dict[taxid])
    i += 1



KeyboardInterrupt: 

## Finally, generate the Anki deck


In [ ]:
my_deck = genanki.Deck(
  2059400110,
  'BIRDS')

my_package = genanki.Package(my_deck)

media_files = os.listdir("downloads")


my_package.media_files = ["downloads/" + i for i in media_files]

with_img = genanki.Model(
  1091735104,
  'Simple Model with Media',
  fields=[
    {'name': 'Answer'},
    {'name': 'MyMedia'},                                  # ADD THIS
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{MyMedia}}',              # AND THIS
      'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
    },
  ])



for file in media_files:
  if file.split(".")[-1] == "png":
    name = file.split(".")[0]
    media = f'<img src="{file}">'  
  elif file.split(".")[-1] == "mp3":
    name = file.split(".")[0]
    media = f'[sound:{file}]'



  note = genanki.Note(
    model = with_img,
    fields=[name,media]
  )

  my_deck.add_note(note)


my_package.write_to_file('output.apkg')